## Contents

* I. Single Sentence Learning ($\alpha=1$)
* II. Multiple Sentence Learning ($\alpha=1$)
* III. $\alpha=0.1$ Initialization

## I. Single Sentence Learning

### A. Evaluation Scheme 1

* Return top 5 most likely property, each accurate prediction accounts for 20% accuracy.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import brown
from spacy.en import English
from collections import defaultdict
from scipy.stats import entropy
from __future__ import division

In [7]:
data_path = "/Users/jacobsw/Desktop/CODER/IMPLEMENTATION_CAMP/BASIC_TOPICS/DISTRIBUTIONAL_SEMANTICS/DATA/McRae-BRM-InPress/"
filename = "CONCS_FEATS_concstats_brm.xls"

In [42]:
df = pd.read_csv(data_path+filename, delimiter='\t')
parser = English()
brown_sents = [unicode(' '.join(sent)) for sent in brown.sents()]
parsed_sents = [parser(sent) for sent in brown_sents]

In [274]:
class Evaluator_Single_Learning:
    
    def __init__(self, parsed_sents, ev_norms, v_argtype_pairs, scheme=1):
        
        self.brown_lemmas, self.brown_t2l = self.make_token2lemma_dict(parsed_sents)
        self.dep_triples = self.extract_dep_triples(parsed_sents)
        
        self.norms = {df.ix[i]['Concept'] for i in range(df.shape[0])}
        self.features = {df.ix[i]['Feature'] for i in range(df.shape[0])}
        self.f2i = {f:i for i,f in enumerate(self.features)}
        self.feature_list = list(self.features) 
        
        self.ev_norms = ev_norms
        self.v_argtype_pairs = v_argtype_pairs
        
        self.scheme = scheme

    ####################
    #### EVALUATION ####
    ####################
    
    def evaluate(self):
        brown_t2l=self.brown_t2l; dep_triples=self.dep_triples
        f2i=self.f2i; feature_list=self.feature_list; norms=self.norms
        ev_norms=self.ev_norms; v_argtype_pairs=self.v_argtype_pairs
        assert len(ev_norms)==len(v_argtype_pairs)
        accuracies = []
        for norm,(v,argtype) in zip(ev_norms,v_argtype_pairs):
            accuracy = self.single_evaluate(brown_t2l, dep_triples, f2i, feature_list, norms,
                                            norm, v, argtype)
            accuracies.append(accuracy)
        
        return accuracies
        
    def single_evaluate(self, brown_t2l, dep_triples, f2i, feature_list, norms, # database
                        out_word, v, argtype): # new input
    
        norms = filter(lambda norm: norm!=out_word, map(lambda norm:self.norm_normalize(norm,brown_t2l),norms))
        norms = map(lambda norm:self.norm_normalize(norm, brown_t2l), norms)
        norms_set = set(norms) 
        norm2prop = self.make_norm2prop(df,brown_t2l)  
        
        P = len(feature_list)
        V = self.training(dep_triples, norm2prop, norms_set, f2i, P) 
        
        w_beta = self.single_update(P, V, v, argtype)
        predicted_props = self.top5_props(v, w_beta, feature_list)
        
        print out_word, predicted_props
        
        if self.scheme==1:
            accuracy = 0
            for prop in predicted_props:
                if prop in norm2prop[out_word]: accuracy += .2
            print "Word: %s | Accuracy: %.6f%%" % (out_word, accuracy*100)
            return accuracy
        else:
            accuracy = 0
            for prop in predicted_props:
                if prop in norm2prop[out_word]: 
                    accuracy += 1
                    print "Word: %s | Accuracy: %.6f%%" % (out_word, accuracy*100)
                    return accuracy
                else: continue
            print "Word: %s | Accuracy: %.6f%%" % (out_word, accuracy*100)
            return accuracy
    
    def single_update(self, P, V, v, argtype): # argtype = {subj, obj}
        w_beta = self.initialize_propdist(P)
        v_alphas = V[v][argtype] # alpha as a list
        for i,alpha_i in enumerate(v_alphas):
            if alpha_i!=1: # i.e. the prop at the position has been seen
                w_beta[i] += alpha_i
        return w_beta 

    def top5_props(self, v, w_beta, feature_list):
        w_beta = w_beta / w_beta.sum()
        avg_ent = entropy(w_beta)
        top5_props_idx = np.argsort(w_beta)[::-1][:5]
        top5_props_names = map(lambda idx:feature_list[idx], top5_props_idx)
        return top5_props_names

    ###################
    #### FUNCTIONS ####
    ###################
    
    def training(self, triples, norm2prop, norms_set, f2i, P):
        V = defaultdict(lambda : defaultdict(lambda : self.initialize_propdist(P)))
        for triple in triples:
            if triple[1].endswith('subj') and triple[0] in norms_set:
                props = self.get_props(norm2prop, triple[0])
                for prop in props:
                    V[triple[2]]['subj'][f2i[prop]] += 1
            elif triple[1].endswith('obj') and triple[0] in norms_set:
                props = self.get_props(norm2prop, triple[0])
                for prop in props:
                    V[triple[2]]['obj'][f2i[prop]] += 1
            else: pass
        return V    
    
    def make_token2lemma_dict(self, parsed_sents):
        lemmas = set()
        token2lemma = {}
        for parsed_sent in parsed_sents:
            for token in parsed_sent:
                token2lemma[token.orth_] = token.lemma_
                lemmas.add(token.lemma_)
        return lemmas, token2lemma
    
    def norm_normalize(self, norm, t2l):
        norm = norm.split('_')[0] if '_' in norm else norm
        if norm in t2l: return t2l[norm]
        return norm    
    
    def make_norm2prop(self, df, t2l):
        norm2prop = defaultdict(list)
        for i in xrange(df.shape[0]):
            norm2prop[self.norm_normalize(df['Concept'][i],t2l)].append(df['Feature'][i]) 
        return norm2prop

    def extract_dep_triples(self, parsed_sents):
        triples = []
        for parsed_sent in parsed_sents:
            for token in parsed_sent:
                lemma_triple = (token.lemma_, token.dep_, token.head.lemma_)
                triples.append(lemma_triple)
        return triples
    
    def initialize_propdist(self, P): 
        return np.zeros(P) + .1
    
    def get_props(self, norm2prop, w):
        return norm2prop[w]
    

In [252]:
import cPickle

In [253]:
path = "/Users/jacobsw/Desktop/UNIV/FALL_2016/LIN389C_RSCH_COMPLING/BAYESIAN/CODE_DRAFTS/DATA/"
norm2mostcommonpair = cPickle.load(open(path+"norm2mostcommonpair.p",'rb'))

In [254]:
norm2mostcommonpair.items()[:5]

[(u'taxi', (u'enter', 'obj')),
 (u'gown', (u'wear', 'obj')),
 (u'bomb', (u'drop', 'obj')),
 (u'kite', (u'fly', 'obj')),
 (u'rocket', (u'build', 'obj'))]

In [255]:
ev_norms = []
v_argtype_pairs = []
for norm,(v,argtype) in norm2mostcommonpair.iteritems():
    ev_norms.append(norm)
    v_argtype_pairs.append((v,argtype))

In [256]:
print len(ev_norms), len(v_argtype_pairs)

317 317


In [257]:
%%time
ev_sg = Evaluator_Single_Learning(parsed_sents, ev_norms, v_argtype_pairs)

CPU times: user 4.74 s, sys: 217 ms, total: 4.96 s
Wall time: 4.92 s


In [259]:
%%time
single_accuracies = ev_sg.evaluate()

In [179]:
print "Single Sentence Learning"
print "Average Accuracy: %.6f%%" % (np.mean(single_accuracies)*100)
print "STD: %.6f" % np.std(single_accuracies)

Single Sentence Learning
Average Accuracy: 10.914826%
STD: 0.196688


### B. Evaluation Scheme 2

* Return top 5 most likely property, record 100% accuracy if either of the 5 is accurate, 0 otherwise.

In [260]:
%%time
ev_sg2 = Evaluator_Single_Learning(parsed_sents, ev_norms, v_argtype_pairs, scheme=2)

CPU times: user 4.77 s, sys: 206 ms, total: 4.98 s
Wall time: 4.94 s


In [263]:
%%time
single_accuracies2 = ev_sg2.evaluate()

In [262]:
print "Single Sentence Learning"
print "Average Accuracy: %.6f%%" % (np.mean(single_accuracies2)*100)
print "STD: %.6f" % np.std(single_accuracies2)

Single Sentence Learning
Average Accuracy: 31.230284%
STD: 0.463433


## II. Multiple Sentence Learning

### A. Evaluation Scheme 1

* See above

In [236]:
from collections import Counter

In [237]:
norm2verbdep = cPickle.load(open(path+"norm2verbdep.p",'rb'))

In [267]:
class Evaluator_Multiple_Learning:
    
    def __init__(self, parsed_sents, norm2verbdep, scheme=1):
        
        self.brown_lemmas, self.brown_t2l = self.make_token2lemma_dict(parsed_sents)
        self.dep_triples = self.extract_dep_triples(parsed_sents)
        
        self.norms = {df.ix[i]['Concept'] for i in range(df.shape[0])}
        self.features = {df.ix[i]['Feature'] for i in range(df.shape[0])}
        self.f2i = {f:i for i,f in enumerate(self.features)}
        self.feature_list = list(self.features) 
        
        self.norm2verbdep = norm2verbdep
        
        self.scheme = scheme

    ####################
    #### EVALUATION ####
    ####################
    
    def evaluate(self):
        brown_t2l=self.brown_t2l; dep_triples=self.dep_triples
        f2i=self.f2i; feature_list=self.feature_list; norms=self.norms
        norm2verbdep = self.norm2verbdep
        accuracies = []
        for norm in norm2verbdep.iterkeys():
            
            accuracy = self.multiple_evaluate(brown_t2l, dep_triples, f2i, feature_list, norms,
                                            norm, norm2verbdep[norm])
            accuracies.append(accuracy)
        
        return accuracies

    
    def multiple_evaluate(self, brown_t2l, dep_triples, f2i, feature_list, norms, # database
                        out_word, v_argtype_pairs): # new input
    
        norms = filter(lambda norm: norm!=out_word, map(lambda norm:self.norm_normalize(norm,brown_t2l),norms))
        norms = map(lambda norm:self.norm_normalize(norm, brown_t2l), norms)
        norms_set = set(norms) 
        norm2prop = self.make_norm2prop(df,brown_t2l)  
        
        P = len(feature_list)
        V = self.training(dep_triples, norm2prop, norms_set, f2i, P) 
        w_beta = self.initialize_propdist(P)
        for v,argtype in v_argtype_pairs:
            w_beta = self.single_update(P, V, w_beta, v, argtype)
        
        predicted_props = self.top5_props(v, w_beta, feature_list)
        
        print out_word, predicted_props
        
        if self.scheme==1:
            accuracy = 0
            for prop in predicted_props:
                if prop in norm2prop[out_word]: accuracy += .2
            print "Word: %s | Accuracy: %.6f%%" % (out_word, accuracy*100)
            return accuracy
        else:
            accuracy = 0
            for prop in predicted_props:
                if prop in norm2prop[out_word]: 
                    accuracy += 1
                    print "Word: %s | Accuracy: %.6f%%" % (out_word, accuracy*100)
                    return accuracy
                else: continue
            print "Word: %s | Accuracy: %.6f%%" % (out_word, accuracy*100)
            return accuracy        

    def single_update(self, P, V, w_beta, v, argtype): # argtype = {subj, obj}
        v_alphas = V[v][argtype] # alpha as a list
        for i,alpha_i in enumerate(v_alphas):
            if alpha_i!=1: # i.e. the prop at the position has been seen
                w_beta[i] += alpha_i
        return w_beta  
    
    def top5_props(self, v, w_beta, feature_list):
        w_beta = w_beta / w_beta.sum()
        avg_ent = entropy(w_beta)
        top5_props_idx = np.argsort(w_beta)[::-1][:5]
        top5_props_names = map(lambda idx:feature_list[idx], top5_props_idx)
        return top5_props_names

    ###################
    #### FUNCTIONS ####
    ###################
    
    def training(self, triples, norm2prop, norms_set, f2i, P):
        V = defaultdict(lambda : defaultdict(lambda : self.initialize_propdist(P)))
        for triple in triples:
            if triple[1].endswith('subj') and triple[0] in norms_set:
                props = self.get_props(norm2prop, triple[0])
                for prop in props:
                    V[triple[2]]['subj'][f2i[prop]] += 1
            elif triple[1].endswith('obj') and triple[0] in norms_set:
                props = self.get_props(norm2prop, triple[0])
                for prop in props:
                    V[triple[2]]['obj'][f2i[prop]] += 1
            else: pass
        return V    
    
    def make_token2lemma_dict(self, parsed_sents):
        lemmas = set()
        token2lemma = {}
        for parsed_sent in parsed_sents:
            for token in parsed_sent:
                token2lemma[token.orth_] = token.lemma_
                lemmas.add(token.lemma_)
        return lemmas, token2lemma
    
    def norm_normalize(self, norm, t2l):
        norm = norm.split('_')[0] if '_' in norm else norm
        if norm in t2l: return t2l[norm]
        return norm    
    
    def make_norm2prop(self, df, t2l):
        norm2prop = defaultdict(list)
        for i in xrange(df.shape[0]):
            norm2prop[self.norm_normalize(df['Concept'][i],t2l)].append(df['Feature'][i]) 
        return norm2prop

    def extract_dep_triples(self, parsed_sents):
        triples = []
        for parsed_sent in parsed_sents:
            for token in parsed_sent:
                lemma_triple = (token.lemma_, token.dep_, token.head.lemma_)
                triples.append(lemma_triple)
        return triples
    
    def initialize_propdist(self, P): 
        return np.zeros(P) + .1
    
    def get_props(self, norm2prop, w):
        return norm2prop[w]
    

In [239]:
%%time
ev_ml = Evaluator_Multiple_Learning(parsed_sents, norm2verbdep)

CPU times: user 4.9 s, sys: 175 ms, total: 5.07 s
Wall time: 5.05 s


In [264]:
%%time
multiple_accuracies = ev_ml.evaluate()

In [242]:
print "Multiple Sentence Learning"
print "Average Accuracy: %.6f%%" % (np.mean(multiple_accuracies)*100)
print "STD: %.6f" % np.std(multiple_accuracies)

Multiple Sentence Learning
Average Accuracy: 16.340694%
STD: 0.211764


### B. Evaluation Scheme 2

* See above

In [268]:
%%time
ev_ml2 = Evaluator_Multiple_Learning(parsed_sents, norm2verbdep, scheme=2)

CPU times: user 4.7 s, sys: 154 ms, total: 4.85 s
Wall time: 4.84 s


In [273]:
%%time
multiple_accuracies2 = ev_ml2.evaluate()

In [270]:
print "Multiple Sentence Learning"
print "Average Accuracy: %.6f%%" % (np.mean(multiple_accuracies2)*100)
print "STD: %.6f" % np.std(multiple_accuracies2)

Multiple Sentence Learning
Average Accuracy: 48.895899%
STD: 0.499878


## III. $\alpha=0.1$ Initialization

### A. Single Sentence Learning

In [275]:
%%time
ev_sg21 = Evaluator_Single_Learning(parsed_sents, ev_norms, v_argtype_pairs)

CPU times: user 4.71 s, sys: 223 ms, total: 4.93 s
Wall time: 4.88 s


In [278]:
%%time
single_accuracies21 = ev_sg21.evaluate()

In [277]:
print "Single Sentence Learning"
print "Average Accuracy: %.6f%%" % (np.mean(single_accuracies21)*100)
print "STD: %.6f" % np.std(single_accuracies21)

Single Sentence Learning
Average Accuracy: 10.914826%
STD: 0.196688


In [279]:
%%time
ev_sg22 = Evaluator_Single_Learning(parsed_sents, ev_norms, v_argtype_pairs, scheme=2)

CPU times: user 4.86 s, sys: 192 ms, total: 5.05 s
Wall time: 5.05 s


In [289]:
%%time
single_accuracies22 = ev_sg22.evaluate()

In [281]:
print "Single Sentence Learning"
print "Average Accuracy: %.6f%%" % (np.mean(single_accuracies22)*100)
print "STD: %.6f" % np.std(single_accuracies22)

Single Sentence Learning
Average Accuracy: 31.230284%
STD: 0.463433


### B. Multiple Sentence Learning

In [282]:
%%time
ev_ml21 = Evaluator_Multiple_Learning(parsed_sents, norm2verbdep)

CPU times: user 4.25 s, sys: 184 ms, total: 4.44 s
Wall time: 4.39 s


In [288]:
%%time
multiple_accuracies21 = ev_ml21.evaluate()

In [284]:
print "Multiple Sentence Learning"
print "Average Accuracy: %.6f%%" % (np.mean(multiple_accuracies21)*100)
print "STD: %.6f" % np.std(multiple_accuracies21)

Multiple Sentence Learning
Average Accuracy: 16.340694%
STD: 0.211764


In [285]:
%%time
ev_ml22 = Evaluator_Multiple_Learning(parsed_sents, norm2verbdep, scheme=2)

CPU times: user 4.51 s, sys: 174 ms, total: 4.68 s
Wall time: 4.66 s


In [290]:
%%time
multiple_accuracies22 = ev_ml22.evaluate()

In [287]:
print "Multiple Sentence Learning"
print "Average Accuracy: %.6f%%" % (np.mean(multiple_accuracies22)*100)
print "STD: %.6f" % np.std(multiple_accuracies22)

Multiple Sentence Learning
Average Accuracy: 48.895899%
STD: 0.499878
